## First and Second derivatives
1-dim function, f(x) : Newton Solve and Optimize

## Gradient, Hessian and Jacobian
N-dim scalar function, f(x,y,z) and vector function, f(f0(x,y),f1(x,y)): Newton Solve and Optimize 

for the InverseJacobian
http://fourier.eng.hmc.edu/e176/lectures/NM/node21.html

Most of ideas of this notebook are from

Massimo Di Pierro 

Annotated Algorithms in Python 

https://mdipierro.github.io/DePaul/CSC402/csc402-notes.pdf

Chapter 4

I modified the code to create an independent python notebook

In [29]:
from numpy import linalg as LA
import numpy as np

### One dimension: Solver and Optimization

In [1]:
# We are preparing 1st and second numerical derivatives for a one dimension f(x)
def D(f , h=1e-6): # first derivative of f
    return lambda x,f=f,h=h: (f(x+h)-f(x-h))/2.0/h

def DD(f, h=1e-6): # second derivative of f
    return lambda x,f=f,h=h: (f(x+h)-2.0*f(x)+f(x-h))/(h*h)

def f(x): 
    return (x-2)*(x-5) 

x = 1.0
print (f(x)) 
f1 = D(f)  # first derivative 
print (f1(x)) 
print (D(f)(x)) 
f2 = DD(f)  # second derivative 
print (DD(f)(x)) 
ff2 = D(f)  # second derivative 
print (D(ff2)(x)) 


4.0
-5.000000000254801
-5.000000000254801
2.001065979584382
2.000399845769607


In [3]:
# solve_newton will allow us to find that point x0 where f(x0) is 0!
def solve_newton(f, x, ap=1e-6, rp=1e-4, ns=20):
    x = float(x) # make sure it is not int
    for k in range(ns):
        (fx, Dfx) = (f(x), D(f)(x))
        if LA.norm(Dfx) < ap:
            raise ArithmeticError('unstable solution')
        (x_old, x) = (x, x-fx/Dfx)
        if (k>2) & (LA.norm(x-x_old) < max(float(ap),LA.norm(x)*rp)): return x
    raise ArithmeticError('no convergence')

In [4]:
sol = solve_newton(f,1.0)
print (sol)
# At the solution point f(x0) is zero!
print (f(sol))
print (round(solve_newton(f,1.0),4))

1.9999999993015118
2.0954644690978615e-09
2.0


In [5]:
# optimize_newton will allow us to find that point x0 where f(x0) is at the minimum (it might be a local minimum)
def optimize_newton(f, x, ap=1e-6, rp=1e-4, ns=20): 
    x = float(x) # make sure it is not int 
    # **************************************************
    # the following line is NOT in solve_netwon    
    (f, Df) = (D(f), DD(f)) # with the optimizer, I set f = D(f) and D(f) = DD(f)
    # **************************************************
    for k in range(ns): 
        (fx, Dfx) = (f(x), Df(x)) 
        if Dfx==0: return x 
        if LA.norm(Dfx) < ap: 
            raise ArithmeticError('unstable solution') 
        (x_old, x) = (x, x-fx/Dfx) 
        if LA.norm(x-x_old) < max(ap,LA.norm(x)*rp): return x 
    raise ArithmeticError('no convergence')


In [6]:
x = 1.0
print (f(x)) 
print (D(f)(x)) 
print (DD(f)(x)) 
print (" testing : ------------")
solOp = optimize_newton(f,1.0)
print (" The solution point, x0, where the function f() finds its minimum is :")
print (solOp)
print (" At the solution point, the function f() is :")
print (f(solOp))
print (" Let's check the gradient at the solution point. It should be 0")
print (D(f)(solOp))

print (" as the second derivative is > 0 at the solution point x0, the solution is a minimum ")
print (DD(f)(solOp)) 

4.0
-5.000000000254801
2.001065979584382
 testing : ------------
 The solution point, x0, where the function f() finds its minimum is :
3.5000000000327596
 At the solution point, the function f() is :
-2.25
 Let's check the gradient at the solution point. It should be 0
0.0
 as the second derivative is > 0 at the solution point x0, the solution is a minimum 
2.000177801164682


### Two dimension: Solver and Optimization

We need, Gradient, Hessian and Jacobian (for a vector function).

In [7]:
# scalar function of three variables, x, y and z
def f(x):
    return 2.0*x[0]+3.0*x[1]+5.0*x[1]*x[2]

In [8]:
def partial(f,i,h=1e-4):
    def df(x,f=f,i=i,h=h):
        x = list(x) # make copy of x
        x[i] += h
        f_plus = f(x)
        x[i] -= 2*h
        f_minus = f(x)
        if isinstance(f_plus,(list,tuple)):
            return [(f_plus[i]-f_minus[i])/(2*h) for i in range(len(f_plus))]
        else:
            return (f_plus-f_minus)/(2*h)
    return df

In [9]:
x = (1,1,1) 
print (round(f(x),4))

10.0


In [10]:
df0 = partial(f,0) 
df1 = partial(f,1) 
df2 = partial(f,2) 

print (df0(x))
print (df1(x))
print (df2(x))

1.9999999999953388
7.999999999999119
4.999999999988347


In [11]:
print (partial(f,0)(x))
print (partial(f,1)(x))
print (partial(f,2)(x))

1.9999999999953388
7.999999999999119
4.999999999988347


In [12]:
def gradient(f, x, h=1e-4):
    v = np.zeros(len(x))
    for i in range(len(x)):
        v[i] = partial(f,i,h)(x)
    return v
grad = gradient(f,x)
print (grad)

def gradient2(f, x, h=1e-4):
    v = np.zeros(len(x))
    for i in range(len(x)):
        df   = partial(f,i,h) 
        v[i] = df(x)
    return v
grad2 = gradient2(f,x)
print (grad2)

def hessian(f, x, h=1e-4):
    s = (len(x),len(x))
    He = np.zeros(s) 
    for i in range(len(x)):
        for j in range(len(x)):
            He[j,i] = partial(partial(f,j,h),i,h)(x)              
    return He
Hess = hessian(f,x)
print (Hess)

[2. 8. 5.]
[2. 8. 5.]
[[0.         0.         0.        ]
 [0.         0.         5.00000006]
 [0.         5.00000006 0.        ]]


In [13]:
# vector function of three variables, x, y and z
def f2(x): 
    return (2.0*x[0]+3.0*x[1]+5.0*x[1]*x[2], 2.0*x[0])

In [14]:
x = (1,1,1) 
print (" Testing : -------------------- ")
print (f2(x))
df0 = partial(f2,0) 
df1 = partial(f2,1) 
df2 = partial(f2,2) 
print (df0(x))
print (df1(x))
print (df2(x))

 Testing : -------------------- 
(10.0, 2.0)
[1.9999999999953388, 1.9999999999997797]
[7.999999999999119, 0.0]
[4.999999999988347, 0.0]


In [15]:
print (" Jacobian Calls : -------------------- ")

def jacobian(f, x, h=1e-4):
    nof = len(f(x))
    nox = len(x)
    s = (nof,nox)
    Jac = np.zeros(s) 
    for i in range(nox):     #colonne
        for j in range(nof): # rows
            ff = partial(f,i,h)(x)
            Jac[j,i] = ff[j]              
    return Jac
Jac = jacobian(f2,x)
print (Jac)
print (type(Jac))
print (Jac.T)

 Jacobian Calls : -------------------- 
[[2. 8. 5.]
 [2. 0. 0.]]
<class 'numpy.ndarray'>
[[2. 2.]
 [8. 0.]
 [5. 0.]]


In [16]:
def solve_newton_multi_Vector_F(f, x, ap=1e-6, rp=1e-4, ns=20): 
    """
    Computes the root of a multidimensional function f near point x.

    Parameters
    f is a function that takes a list and returns a scalar
    x is a list

    Returns x, solution of f(x)=0, as a list
    """
    for k in range(ns): 
        (fx, Dfx) = (f(x), jacobian(f,x)) 
        if LA.norm(Dfx) < ap: 
            raise ArithmeticError('unstable solution')
        Dfx = np.transpose(Dfx)
        # prepare the jacobian pseudo inverse
        Trans = np.transpose(Dfx)
        Pseudo = np.dot(np.linalg.inv(np.dot(Trans, Dfx)), Trans)
        (x_old, x) = (x, x-np.dot(fx,Pseudo)) 
        if LA.norm(x-x_old) < max(ap,LA.norm(x)*rp): return x 
    raise ArithmeticError('no convergence')

def optimize_newton_multi_Scalar_F(f, x, ap=1e-6, rp=1e-4, ns=20): 
    for k in range(ns): 
        (fx, Dfx) = (gradient(f,x), hessian(f,x)) 
        if LA.norm(Dfx) < ap: 
            raise ArithmeticError('unstable solution')  
        (x_old, x) = (x, x-np.dot(fx,np.linalg.inv(Dfx))) 
        if LA.norm(x-x_old) < max(ap,LA.norm(x)*rp): return x 
    raise ArithmeticError('no convergence')

This is for a function returning a 2-dim vector, from R3 to R2. For this function we want to find the point where it returns the zero vector<0,0>

In [17]:
# Remember from R3 to R2
x = (1,1,1) 
print ("at x = <1,1,1>, f(x) is equal to: ")
print (f2(x))

at x = <1,1,1>, f(x) is equal to: 
(10.0, 2.0)


In [18]:
print("--------------------")
# solve on a vector - function means to find the point x,y,z where the vector f(x,y,z) returns the zero vector <0,0>!
sol = solve_newton_multi_Vector_F(f2, x=x)
print ("the point <x, y, z> where f(x,y,z) is = <0, 0> are: ")
print (np.round(sol,4))
print (f2(sol))

--------------------
the point <x, y, z> where f(x,y,z) is = <0, 0> are: 
[0.     0.     0.4856]
(1.7242216261363584e-16, 0.0)


In [19]:
print("--------------------")
print ("At the solution points, the Jacobian is NOT a zero vector/matrix! Remember this is a solve, not optimization. \
Let's check")
print (np.round(jacobian(f2,sol),4))
print("--------------------")
print ("and finally, at the solution point <x, y, z> where f(x,y,z) is = <0, 0>: ")
print (np.round(f2(sol),4))

--------------------
At the solution points, the Jacobian is NOT a zero vector/matrix! Remember this is a solve, not optimization. Let's check
[[2.     5.4281 0.    ]
 [2.     0.     0.    ]]
--------------------
and finally, at the solution point <x, y, z> where f(x,y,z) is = <0, 0>: 
[0. 0.]


This is for a function returning a scalar, from R2 to R1. For this function we want to find the point where it is at its minimum

In [20]:
def f(x): 
    return (x[0]-2)**2+(x[1]-3)**2 
x = (5,5) 

grad = gradient(f,x)
print ("the Gradient of f() at the initial point <5, 5> is: ",grad)

Hess = hessian(f,x)
print ("the Hessian Matrix of f() at the same point <5, 5> is: ") 
print (Hess)

the Gradient of f() at the initial point <5, 5> is:  [6. 4.]
the Hessian Matrix of f() at the same point <5, 5> is: 
[[1.99999999 0.        ]
 [0.         1.99999999]]


In [21]:
print ("Does the Hessian Matrix have an inverse ? ") 
print (np.linalg.inv(Hess))

Does the Hessian Matrix have an inverse ? 
[[0.5 0. ]
 [0.  0.5]]


In [22]:
print("--------------------")
sol = optimize_newton_multi_Scalar_F(f, x=x)
print ("solution is ",sol)
print ("f at solution is ",f(sol))
print ("grad at solution is ")
print (gradient(f,sol))
print ("The gradient is a zero vector, so this is a solution")


--------------------
solution is  [2. 3.]
f at solution is  4.930380657631324e-32
grad at solution is 
[-4.44088455e-16  4.44088455e-16]
The gradient is a zero vector, so this is a solution


In [23]:
print ("-----------------------------------------------------")
print ("Let's look at the Hessian ")
print ("hessian at solution is :")
print(hessian(f,sol))
print ("Is it a minumum? ")
HessSol = hessian(f,sol)
print(HessSol)
print ("----------------------------------------------------- \n")

-----------------------------------------------------
Let's look at the Hessian 
hessian at solution is :
[[ 2.00000000e+00 -8.27180613e-17]
 [-8.27180613e-17  2.00000000e+00]]
Is it a minumum? 
[[ 2.00000000e+00 -8.27180613e-17]
 [-8.27180613e-17  2.00000000e+00]]
----------------------------------------------------- 



In [28]:
print ("To check, let's create a vector v, which when multiplied by H should return a non-zero vector, \
if the point is a minimum")
print ("T(v) * H * v >0 ")
v = np.array([1, 2])

print (np.dot(np.dot(v,HessSol),v))
print (np.dot(np.dot(v,HessSol),np.transpose(v)))
print ("Yes, it is! ")

To check, let's create a vector v, which when multiplied by H should return a non-zero vector, if the point is a minimum
T(v) * H * v >0 
10.000000000000016
10.000000000000016
Yes, it is! 
